С появлением метода опорных векторов и ядерных методов ([1963]($On a perceptron class$), [1992]($A Training Algorithm for Optimal Margin Classifiers$), [1995]($Support-Vector Networks$)) и развитием глубоких нейронных сетей ([1998]($Gradient-Based Learning Applied to Document Recognition$), [2006]($A Fast Learning Algorithm for Deep Belief Nets$), [2009]($Learning Deep Architectures for AI$)) исследователей стала интересовать взаимосвязь между этими классами алгоритмов.

В первой части данной работы предложен класс ядер, названных arc-cosine kernels. Эти ядра имитируют работу полносвязной случайно инциализированной нейронной сети бесконечной ширины, имеющей один или несколько скрытых слоев. Во второй части работы авторы предлагают алгоритм классификации multilayer kernel machines.

О методе опорных векторов и ядерных методах я писал отдельный [обзор]($SVM и ядерные методы в машинном обучении$).

(in progress)